In [61]:
#필요한 라이브러리 호출
import os
import numpy as np
import json
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

In [62]:
def load_segment_data(filepaths):
    segment_data = {}
    for filepath in filepaths:
        npz_file = np.load(filepath, allow_pickle=True)
        for key in npz_file.files:
            normalized_key = os.path.basename(key)  # 파일 이름만 추출하여 키로 사용
            segment_data[normalized_key] = filepath  
    return segment_data

In [63]:
def get_segment_data(filepath, filename):
    npz_file = np.load(filepath, allow_pickle=True)
    return npz_file[filename]

In [64]:
segment_paths = {
    'anger': [
        'segmentation/train/train_anger.npz',
        'segmentation/val/val_anger.npz'
    ],
    'happy': [
        'segmentation/train/train_happy.npz',
        'segmentation/val/val_happy.npz'
    ],
    'panic': [
        'segmentation/train/train_panic.npz',
        'segmentation/val/val_panic.npz'
    ],
    'sadness': [
        'segmentation/train/train_sadness.npz',
        'segmentation/val/val_sadness.npz'
    ]
}

segment_data = {emotion: load_segment_data(paths) for emotion, paths in segment_paths.items()}


In [65]:
TRAIN_IMG_PATH = 'img/img/train/'
VALID_IMG_PATH = 'img/img/val/'

TRAIN_JSON_PATH = 'label/train/'
VALID_JSON_PATH = 'label/val/'

OUTPUT_PATH = 'test_img/'

In [66]:
def load_labels(json_path):
    with open(json_path, "r", encoding='euc-kr') as f:
        return json.load(f)

train_label_data = {
    'anger': load_labels(TRAIN_JSON_PATH + "train_anger.json"),
    'happy': load_labels(TRAIN_JSON_PATH + "train_happy.json"),
    'panic': load_labels(TRAIN_JSON_PATH + "train_panic.json"),
    'sadness': load_labels(TRAIN_JSON_PATH + "train_sadness.json")
}

valid_label_data = {
    'anger': load_labels(VALID_JSON_PATH + "val_anger.json"),
    'happy': load_labels(VALID_JSON_PATH + "val_happy.json"),
    'panic': load_labels(VALID_JSON_PATH + "val_panic.json"),
    'sadness': load_labels(VALID_JSON_PATH + "val_sadness.json")
}



In [67]:
def mask_img(filename, path, segment):
    image_path = os.path.join(path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    new_img = image.copy()
    # 이미지 face 만 남기고 나머지는 가림
    new_img[segment == 0] = 0
    new_img[segment == 1] = 0
    new_img[segment == 2] = 0
    new_img[segment == 4] = 0
    new_img[segment == 5] = 0

    return new_img

In [68]:
def crop_face(label_data, img):

  boxes = [label_data['annot_A']['boxes'], label_data['annot_B']['boxes'], label_data['annot_C']['boxes']]
  avg_box = { # annot_A/B/C 각 좌표의 평균 값 구해서 저장
      'maxX': np.mean([box['maxX'] for box in boxes]),
      'maxY': np.mean([box['maxY'] for box in boxes]),
      'minX': np.mean([box['minX'] for box in boxes]),
      'minY': np.mean([box['minY'] for box in boxes])
  }

  new_image = img[int(avg_box['minY']):int(avg_box['maxY']), int(avg_box['minX']):int(avg_box['maxX'])] # 평균값으로 이미지 crop

  return new_image

In [73]:

def find_label_data(filename, label_data):
    for item in label_data:
        if item['filename'] == filename:
            return item
    return None

# 이미지 처리 함수
def process_images(emotion, input_folder, output_folder, segment_data, label_data):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                filename = os.path.basename(file)
                filepath = segment_data.get(filename)
                if filepath is None:
                    continue

                segment = get_segment_data(filepath, filename)
                
                json_data = find_label_data(filename, label_data)
                if json_data is None:
                    continue

                masked_img = mask_img(filename, input_folder, segment)
                if masked_img is None:
                    continue

                cropped_img = crop_face(json_data, masked_img)
                if cropped_img is None:
                    continue

                output_path = os.path.join(output_folder, emotion)
                if not os.path.exists(output_path):
                    os.makedirs(output_path)

                output_file_path = os.path.join(output_path, filename)
                if cropped_img.size > 0:
                    cv2.imwrite(output_file_path, cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR))
                else:
                    print("Error")

In [74]:
for emotion in segment_data.keys():
    train_input_folder = os.path.join(TRAIN_IMG_PATH, emotion)
    valid_input_folder = os.path.join(VALID_IMG_PATH, emotion)
    train_output_folder = os.path.join(OUTPUT_PATH, 'train', emotion)
    valid_output_folder = os.path.join(OUTPUT_PATH, 'val', emotion)

    process_images(emotion, train_input_folder, train_output_folder, segment_data[emotion], train_label_data[emotion])
    process_images(emotion, valid_input_folder, valid_output_folder, segment_data[emotion], valid_label_data[emotion])

Error
Error
Error
Error
Error
